# Industry visualization
This project is all about helping users understand the charactistics of a given industry and understand them in the context of all industries. It mainly uses Prof. Damodaran's data. Only publicly traded companies in the industry are considered. 

In [1]:
# Import required libraries
import pandas as pd
import numpy as np

import sys
import os

sys.path.append("../")
from edgar_utils import ticker_to_cik, get_companywide_concepts, get_financial_item
from adamodaran_utils import *

from pprint import pprint
pd.set_option('display.width', 1000)

In [30]:
ind_list = get_industry_list()
filtered_list = [s for s in ind_list if s.lower().startswith('computer')]
print(filtered_list)

['Computer Services', 'Computers/Peripherals']


In [8]:
ind_df = get_industry_df()
pprint(ind_df.head())

                                                 company     industry                  sector  SIC Code        Country  exchange ticker
41022                The Trade Desk, Inc. (NasdaqGM:TTD)  Advertising  Communication Services      7370  United States  NasdaqGM    TTD
41023                      Omnicom Group Inc. (NYSE:OMC)  Advertising  Communication Services      7311  United States      NYSE    OMC
41024  The Interpublic Group of Companies, Inc. (NYSE...  Advertising  Communication Services      7311  United States      NYSE    IPG
41025                      Magnite, Inc. (NasdaqGS:MGNI)  Advertising  Communication Services      7310  United States  NasdaqGS   MGNI
41026                           Ibotta, Inc. (NYSE:IBTA)  Advertising  Communication Services      7310  United States      NYSE   IBTA


In [13]:
print(ind_df[ind_df.industry=="Apparel"])

                                                company industry                  sector  SIC Code        Country  exchange ticker
41167                         Tapestry, Inc. (NYSE:TPR)  Apparel  Consumer Discretionary      3100  United States      NYSE    TPR
41168                Ralph Lauren Corporation (NYSE:RL)  Apparel  Consumer Discretionary      2300  United States      NYSE     RL
41169                       V.F. Corporation (NYSE:VFC)  Apparel  Consumer Discretionary      2300  United States      NYSE    VFC
41170                    Levi Strauss & Co. (NYSE:LEVI)  Apparel  Consumer Discretionary      2300  United States      NYSE   LEVI
41171                              PVH Corp. (NYSE:PVH)  Apparel  Consumer Discretionary      2300  United States      NYSE    PVH
41172       Columbia Sportswear Company (NasdaqGS:COLM)  Apparel  Consumer Discretionary      2300  United States  NasdaqGS   COLM
41173                   Kontoor Brands, Inc. (NYSE:KTB)  Apparel  Consumer Discreti

In [7]:
# company_ticker = "CMG"
company_ticker = "LULU"
indname = ind_df[ind_df['ticker']==company_ticker].industry.values[0]
print("Industry of {}: {}".format(company_ticker, indname))
print("----------------------------")
companies = ind_df[ind_df['industry']==indname].company.values
# print("\n")
print("List of companies in this industry:")
print("----------------------------")
print(companies)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [14]:
# indname = 'Total Market (without financials)'
indname = 'Apparel'
demand_df = get_ind_demand(industry_list=[indname])
fundamentals_df = get_ind_fundamentals(industry_list=[indname])
profitability_df = get_ind_profitability(industry_list=[indname])
efficiency_df = get_ind_efficiency(industry_list=[indname])
risk_df = get_ind_risk(industry_list=[indname])
multiples_df = get_ind_multiples(industry_list=[indname])

In [ ]:
print("-----------------------------------------")
print(" {} INDUSTRY".format(indname.upper()))
print("-----------------------------------------")

s1 = "Market_size: ${} B".format(fundamentals_df.loc[indname,'Revenue'])
s2 = "Past CAGR (5y): {}%".format(demand_df.loc[indname, 'Revenue CAGR (past 5y)'])
s3 = "Next CAGR (2y): {}%".format(demand_df.loc[indname, 'Revenue CAGR (next 2y)'])
s4 = "Next CAGR (5y): {}%".format(demand_df.loc[indname, 'Revenue CAGR (next 5y)'])
s5 = ""

ps1 = "Gross Margin: {}%".format(profitability_df.loc[indname,'Gross Margin'])
ps2 = "EBIT Margin: {}%".format(profitability_df.loc[indname,'EBIT Margin'])
ps3 = "Net Margin: {}%".format(profitability_df.loc[indname,'Net Margin'])
ps4 = "ROC: {}%".format(profitability_df.loc[indname,'ROC'])
ps5 = "ROE: {}%".format(profitability_df.loc[indname,'ROE'])

es1 = "Receivable days: {}".format(np.int64(efficiency_df.loc[indname,'DSO']))
es2 = "Inventory days: {}".format(np.int64(efficiency_df.loc[indname,'DSI']))
es3 = "Payable days: {}".format(np.int64(efficiency_df.loc[indname,'DPO']))
es4 = ""
es5 = ""

rs1 = "No. of firms: {}".format(risk_df.loc[indname,'Number of firms'])
rs2 = "Beta: {}".format(risk_df.loc[indname,'Beta'])
rs3 = "D/E: {}".format(risk_df.loc[indname,'D/E'])
rs4 = ""
rs5 = ""

ms1 = "Current PE: {}".format(multiples_df.loc[indname,'Current PE'])
ms2 = "Forward PE: {}".format(multiples_df.loc[indname,'Forward PE'])
ms3 = "EV/EBITDA: {}".format(multiples_df.loc[indname,'EV/EBITDA'])
ms4 = "PBV: {}".format(multiples_df.loc[indname,'PBV'])
ms5 = "EV/Sales: {}".format(multiples_df.loc[indname,'EV/Sales'])


display_df = pd.DataFrame({
                            'MARKET':[s1,s2,s3,s4,s5],
                            'PROFITABILITY':[ps1,ps2,ps3,ps4,ps5],
                            'EFFICIENCY':[es1,es2,es3,es4,es5],
                            'RISK':[rs1,rs2,rs3,rs4,rs5],
                            'MULTIPLES':[ms1,ms2,ms3,ms4,ms5]
                           })
pprint(display_df)

-----------------------------------------
 RESTAURANT/DINING INDUSTRY
-----------------------------------------
                MARKET      PROFITABILITY           EFFICIENCY              RISK          MULTIPLES
0  Market_size: $168 B  Gross Margin: 33%  Receivable days: 18  No. of firms: 62  Current PE: 154.9
1   Past CAGR (5y): 9%   EBIT Margin: 13%   Inventory days: 11         Beta: 1.1   Forward PE: 38.9
2  Next CAGR (2y): 10%    Net Margin: 11%     Payable days: 24         D/E: 23.0    EV/EBITDA: 18.7
3  Next CAGR (5y): 17%           ROC: 20%                                                  PBV: nan
4                                 ROE: 0%                                             EV/Sales: 4.5
